In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# from pathlib import Path
# import torch

# DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
# torch.manual_seed(1337)

# import v5_practice1 as vp

# # with Path("/root/language-modeling/input.txt").open("r", encoding="utf-8") as f:
# with Path("../input.txt").open("r", encoding="utf-8") as f:
#     text = f.read()

# block_size = 8
# batch_size = 32
# n_heads = 4
# n_embd = 32
# max_iters = 4000
# eval_iters = 200

# data = vp.Data(text)
# data.block_size = block_size
# data.batch_size = batch_size
# data.n_embd = n_embd

# # print("".join(data.decode(data.encode("hi there"))))
# # xb, yb = data.get_batch('train')
# # print(xb, yb)

# # m = vp.LanguageModel(n_embd, block_size, data.vocab_size, n_heads)
# # out = m.generate(torch.zeros((1, 1), dtype=torch.long))
# # # out
# # print("".join(data.decode(out[0].tolist())))

# m = vp.LanguageModel(n_embd, block_size, data.vocab_size, n_heads)
# m = m.to(DEVICE)
# optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)
# for iter_ in range(max_iters):
#     if iter_ % eval_iters == 0:
#         out = data.estimate_loss(m, eval_iters)
#         print(f"train loss {out['train']} val loss {out['val']}")
#     xb, yb = data.get_batch('train')
#     logits, losses = m(xb, yb)
#     optimizer.zero_grad()
#     losses.backward()
#     optimizer.step()

# out = m.generate(torch.zeros((1, 1), dtype=torch.long))
# print("".join(data.decode(out[0].tolist())))

In [3]:
# !pip list | grep torch

In [21]:
import v5_practice2 as vp

In [47]:
vp.main()

train loss 4.188170909881592 val loss 4.188495635986328
train loss 3.3207733631134033 val loss 3.3166937828063965
train loss 3.2974908351898193 val loss 3.2898130416870117
train loss 3.254143714904785 val loss 3.2516660690307617
train loss 3.149306058883667 val loss 3.1776113510131836
train loss 3.0998528003692627 val loss 3.161972761154175
train loss 3.0346269607543945 val loss 3.093740463256836
train loss 3.0084850788116455 val loss 3.0409040451049805
train loss 2.963918447494507 val loss 3.06240177154541
train loss 2.9350388050079346 val loss 3.0095555782318115
train loss 2.861933946609497 val loss 2.985308885574341
train loss 2.8761937618255615 val loss 2.941310167312622
train loss 2.783820390701294 val loss 2.8646750450134277
train loss 2.7275478839874268 val loss 2.810793876647949
train loss 2.711230754852295 val loss 2.7707583904266357
train loss 2.65073299407959 val loss 2.7967586517333984
train loss 2.6412253379821777 val loss 2.767009973526001
train loss 2.6432788372039795 va